In [1]:
# 공식 웹사이트에서 설치해야 함
from elasticsearch import Elasticsearch, helpers
import difflib

# Elasticsearch 클라이언트 설정
es = Elasticsearch("http://localhost:9200")

# 인덱스 생성
index_name = "dental_claims"
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

# 데이터 색인
data = {
    "incomplete_disease_code": [
        "K00.0", "K00.1", "K01.0", "K02.0", "K02.1", 
        "K03.0", "K03.1", "K04.0", "K05.0", "K05.1",
        "K06.0", "K06.1", "K07.0", "K08.0", "K09.0",
        "K10.0", "K11.0", "K12.0", "K13.0", "K14.0"
    ],
    "incomplete_disease_name": [
        "선천성 구강병", "선천성 악질환", "유치의 충치", "상악의 감염", "매복 사랑니",
        "치주염", "치은염", "전치부 충치", "상악 치조골의 질병", "하악의 감염",
        "구강내 염증", "입안의 종양", "구강의 기타 질환", "턱관절 장애", "구강 궤양",
        "급성 치통", "치근단 염증", "악골의 낭종", "하악 골절", "상악 골절"
    ],
    "claim_category": [
        "진료", "수술", "진료", "치료", "수술",
        "치료", "치료", "진료", "진료", "수술",
        "치료", "치료", "치료", "수술", "진료",
        "진료", "수술", "치료", "진료", "진료"
    ],
    "claim_detail": [
        "구강 검진", "사랑니 발치", "충치 치료", "치아 발치", "매복 사랑니 발치",
        "치주 치료", "치은 치료", "전치부 충치 치료", "치조골 치료", "하악 감염 치료",
        "구강 염증 치료", "종양 제거", "기타 치료", "턱관절 치료", "구강 궤양 치료",
        "급성 치통 치료", "치근단 염증 치료", "악골 낭종 제거", "하악 골절 치료", "상악 골절 치료"
    ]
}

# 색인 데이터 추가
def index_data(data):
    actions = [
        {
            "_index": index_name,
            "_source": {
                "incomplete_disease_code": code,
                "incomplete_disease_name": name,
                "claim_category": category,
                "claim_detail": detail
            }
        }
        for code, name, category, detail in zip(
            data["incomplete_disease_code"],
            data["incomplete_disease_name"],
            data["claim_category"],
            data["claim_detail"]
        )
    ]
    helpers.bulk(es, actions)

index_data(data)

# 오타 교정 기능
def correct_spelling(query):
    all_diseases = data["incomplete_disease_name"]
    matches = difflib.get_close_matches(query, all_diseases)
    return matches[0] if matches else query

# 검색 함수
def search_claims(natural_query):
    # 오타 교정
    corrected_query = correct_spelling(natural_query)

    # 검색 쿼리 생성
    body = {
        "query": {
            "multi_match": {
                "query": corrected_query,
                "fields": ["incomplete_disease_name", "claim_detail"]
            }
        }
    }
    
    results = es.search(index=index_name, body=body)
    
    return [
        {
            "code": hit["_source"]["incomplete_disease_code"],
            "name": hit["_source"]["incomplete_disease_name"],
            "category": hit["_source"]["claim_category"],
            "detail": hit["_source"]["claim_detail"]
        }
        for hit in results["hits"]["hits"]
    ]

# 테스트 쿼리
queries = [
    "사랑니 발치했어요.",
    "충치 치료는 어떻게 하나요?", 
    "턱관절이 아프다.",
    "사악",
    "구강 염증 치료가 필요해요.", 
    "발치가 필요할까요?", 
    "치료 방법은 뭐가 있을까?", 
    "매복 사랑니가 너무 아파요.", 
    "구강 검진 예약하고 싶어요.", 
    "종양 제거 수술",
    "치주염 치료는 어디서 받지?",
    "K01.0",
    "치아",
    "치주 치료",
    "사ㅏ랑니",
    "cldk"
]

# 각 쿼리로 검색 수행
for query in queries:
    results = search_claims(query)
    print(f"Query: {query}")
    for result in results:
        print(f"Code: {result['code']}, Name: {result['name']}, Category: {result['category']}, Detail: {result['detail']}")
    print("===")

KeyboardInterrupt: 

1. Elasticsearch 설치: 아직 설치하지 않았다면 Elasticsearch 공식 웹사이트에서 설치합니다.
(https://www.elastic.co/guide/en/elasticsearch/reference/current/install-elasticsearch.html)

2. Elasticsearch 실행:

- 설치 후 Elasticsearch 디렉토리에서 bin/elasticsearch (Windows에서는 bin/elasticsearch.bat) 명령어를 통해 실행합니다.

3. 응답 확인: 실행 후 브라우저에서 http://localhost:9200로 접속하여 다음과 같은 JSON 응답이 나타나는지 확인합니다.

```json
코드 복사
{
  "name" : "your-node-name",
  "cluster_name" : "your-cluster-name",
  "cluster_uuid" : "your-cluster-uuid",
  "version" : {
    "number" : "x.y.z",
    "build_flavor" : "default",
    "build_type" : "zip",
    "build_hash" : "xxxxxxxx",
    "build_date" : "yyyy-mm-dd",
    "build_snapshot" : false,
    "lucene_version" : "x.y.z",
    "minimum_wire_compatibility_version" : "x.y.z",
    "minimum_index_compatibility_version" : "x.y.z"
  },
  "tagline" : "You Know, for Search"
}
```